In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import joblib
import requests

In [51]:
# call api
url = "https://api.weirdgloop.org/exchange/history/rs/all?id=2"

response = requests.get(url)
data = response.json()

# Extract the list from the correct key (item ID)
item_id = "2"
if item_id in data:
    df = pd.DataFrame(data[item_id])  # Convert to DataFrame
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")  # Convert timestamp from milliseconds
    print(df.head())
else:
    print(f"Item ID {item_id} not found in response.")

  id  price  volume  timestamp
0  2    188     NaN 2008-05-21
1  2    186     NaN 2008-05-22
2  2    186     NaN 2008-05-23
3  2    184     NaN 2008-05-24
4  2    184     NaN 2008-05-25


In [52]:
# create features
df["prev_price"] = df["price"].shift(1)
X = df[["prev_price"]]  # features
y = df["price"]  # target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [53]:
# train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

Mean Absolute Error: 8.169253473106481


In [54]:
# save model
joblib.dump(model, "runescape_model.pkl")

['runescape_model.pkl']

In [55]:
# load model
model = joblib.load("runescape_model.pkl")

In [56]:
# test prediction
latest_price = df["prev_price"].iloc[-1]

next_input = pd.DataFrame([[latest_price]], columns=["prev_price"])

predicted_price = model.predict(next_input)

print(f"Predicted Next Price: {predicted_price[0]}")

Predicted Next Price: 1511.8491666666669
